In [ ]:
import re
data = []

# request page may pha ca phe
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver-manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument('-headless') 
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("--incognito")
i = 1
while True:
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.implicitly_wait(30)

  tiki_url = f'https://tiki.vn/may-pha-ca-phe/c1939?page={i}&src=c.1882.hamburger_menu_fly_out_banner'
  print('this is -----------------------------------------------',tiki_url)
  driver.get(tiki_url)

  html_data = driver.page_source

  driver.close()
    
  # parse html into beautiful soup
  from bs4 import BeautifulSoup

  # r.text is a HTML file so use html.parser
  soup = BeautifulSoup(html_data, 'html.parser')
  # total number of items:  
  result_items = soup.find('div', {'class':"CategoryViewstyle__Right-bhstkd-1 jJGphj"}).div.div.div.h4.text.split()[0]
  # scripts of all item in page 1     
  scripts = soup.find_all('script', {'type': 'application/ld+json'})

  if int(len(data)) >= int(result_items):
    break
  else:
    for a in range(1,len(scripts)-1): 
      test = str(scripts[a])

      # find product_id
      pattern = r'.*"sku":"(\d+)".*'
      product_id1 = str(re.findall(pattern, test)).strip('[')
      product_id2 = product_id1.strip(']')
      product_id = product_id2.strip("'")

      # file title
      pattern = r'"name":"(.*)","des'
      title1 = str(re.findall(pattern, test)).strip('[')
      title2 = title1.strip(']')
      title = title2.strip("'")

      # find price
      pattern = r'"price":(\d+)}'
      price1 = str(re.findall(pattern, test)).strip('[')
      price2 = price1.strip(']')
      price = price2.strip("'")

      # find image
      pattern = r'"image":"(.*g)","name'
      image_url1 = str(re.findall(pattern, test)).strip('[')
      image_url2 = image_url1.strip(']')
      image_url = image_url2.strip("'")

      # find product_url
      pattern = r'"url":"(.*)","image"'
      product_url1 = str(re.findall(pattern, test)).strip('[')
      product_url2 = product_url1.strip(']')
      product_url = product_url2.strip("'")

      # create data contain dict of 48 items
      d_script = {'product_id':'', 'title':'', 'price':'', 'image_url':'', 'product_url':''}
      try:
          d_script['product_id'] = product_id
          d_script['title'] = title
          d_script['price'] = price
          d_script['image_url'] = image_url
          d_script['product_url'] = product_url
          data.append(d_script)

      except:
          print("We got one article error!")
  i += 1


     |████████████████████████████████| 911kB 5.5MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 30 not upgraded.
Need to get 83.2 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 89.0.4389.90-0ubuntu0.18.04.2 [1,127 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 89.0.4389.90-0ubuntu0.18.04.2 [73.6 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd

In [ ]:
len(data)

196

In [ ]:
# create dataframe
import pandas as pd
 
product = pd.DataFrame(data = data)  

In [ ]:
product

,product_id,title,price,image_url,product_url
0,5756585384755,Máy Pha Cafe Tự Động Siemens TQ503D01 EQ.500 ...,22990000,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tiki.vn/may-pha-cafe-tu-dong-siemens-t...
1,1777835653808,Máy Pha Cà Phê Espresso Tiross TS-621 - Hàng C...,899000,https://salt.tikicdn.com/cache/280x280/media/c...,https://tiki.vn/may-pha-ca-phe-espresso-tiross...
2,8966937512332,Máy pha cà phê Espresso Zamboo ZB90-PRO - Hàng...,2880000,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tiki.vn/may-pha-ca-phe-espresso-zamboo...
3,5457228262997,Máy Pha Cà Phê Espresso Delonghi ECP31.21 (110...,3689000,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tiki.vn/may-pha-ca-phe-espresso-delong...
4,3608228432387,[Hàng Nhập khẩu Đức] Dung dịch khử cặn ấm nước...,70000,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tiki.vn/hang-nhap-khau-duc-dung-dich-k...
...,...,...,...,...,...
191,8914419017189,Máy xay cafe mini cầm tay siêu nhanh,262561,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tiki.vn/may-xay-cafe-mini-cam-tay-sieu...
192,6026901571238,Máy xay cafe cầm tay mọi lúc mọi nơi (màu nâu),350000,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tiki.vn/may-xay-cafe-cam-tay-moi-luc-m...
193,1774779944764,Máy Pha Cà Phê Sinbo SCM-2938 - Đen - Hàng chí...,479000,https://salt.tikicdn.com/cache/280x280/media/c...,https://tiki.vn/may-pha-ca-phe-sinbo-scm-2938-...
194,4749137688830,Máy Tạo Bọt Sữa CT Trung Nguyên Legendccino II...,2723400,https://salt.tikicdn.com/cache/280x280/ts/prod...,https://tiki.vn/may-tao-bot-sua-ct-trung-nguye...
